# Análisis predictivo de series temporales
------

## Análisis exploratorio, regresión lineal y no lineal.

In [ ]:
#install.packages("devtools")     # only need to do this once
#devtools::install_github("nickpoison/astsa")
#install.packages("astsa")
#library(astsa)
options(repr.plot.width=8, repr.plot.height=7) #ajusta tamaño de graficas

## Ejercicio 1 (Prob 2.1 del libro):

Para los datos de Johnson & Johnson (`jj`), considerados como serie $y_t$, considere la transformación $x_t = \log(y_t)$. Dicha transformación muestra los incrementos relativos o porcentuales (al pasar a logaritmo).

1. Ajustar un modelo de la forma:
$$x_t = \beta t + \sum_{i=1}^4 \alpha_i Q_i(t) + w_t$$
siendo $Q_i$ una función que vale $1$ si se está en el trimestre $i$ y $0$ si no.

2. Graficar los datos y superponer los datos ajustados.

3. Calcular los residuos y evaluar el ajuste.

In [244]:
x_t = (log(jj)) -1970
tsplot(x_t, main = "JJ", col = 8)

In [245]:
t = time(x_t)
Q = factor(cycle(jj) ) # make (Q)uarter factors

In [246]:
fit = lm(x_t ~ 0 +  t + Q , na.action=NULL) # no intercept

In [247]:
summary(fit) # view the results (not shown)


Call:
lm(formula = x_t ~ 0 + t + Q, na.action = NULL)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.29318 -0.09062 -0.01180  0.08460  0.27644 

Coefficients:
     Estimate Std. Error t value Pr(>|t|)    
t   1.672e-01  2.259e-03    74.0   <2e-16 ***
Q1 -2.298e+03  4.451e+00  -516.4   <2e-16 ***
Q2 -2.298e+03  4.451e+00  -516.3   <2e-16 ***
Q3 -2.298e+03  4.452e+00  -516.3   <2e-16 ***
Q4 -2.298e+03  4.452e+00  -516.2   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1254 on 79 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 4.143e+09 on 5 and 79 DF,  p-value: < 2.2e-16


In [248]:
AIC(fit)
BIC(fit)


[1] -103.6153

[1] -89.03043

In [249]:
predictions = ts(fitted(fit), start = 1960, frequency = 4) #Creo una serie temporal con los valores ajustados.
tsplot(x_t, col = 8)
lines(predictions, col = 3)

In [250]:
coef(fit)

t            Q1            Q2            Q3            Q4 
    0.1671722 -2298.2763707 -2298.2482479 -2298.1781396 -2298.4468973

In [251]:
tsplot(residuals(fit), main = "Residuos", col = 8)

In [252]:
acf(residuals(fit))

In [253]:
qqnorm(residuals(fit), col = 3)
qqline(residuals(fit), col = 8)

Analizando el comportamiento de los lags de la ACF, se observa que aùn no se ha logrado capturar toda la informaciòn del modelo. Aùn existe una correlaciòn de los errores con os años anteriores. 

Tambien al observar el comportamiento Normal de los residuos, podemos decir que el modelo no se ajusta completamente. 

## Ejercicio 2 (ajuste de una señal):

Considere la señal generada más abajo, que consiste en una componente periódica más ruido. 

1. Ajuste un modelo de la forma:
$$x_t = \beta_1 + \beta_2 \cos(2\pi t) + \beta_3 \sin(2\pi t).$$

2. Ajuste un modelo de la forma:
$$x_t = \beta_1 + \beta_2 \cos(2\pi t) + \beta_3 \sin(2\pi t) + \beta_4 \cos(2*\pi 2t) + \beta_5 \sin(2\pi 2t).$$
¿Qué captura este segundo modelo?

3. Compare los residuos y los ajustes en cada uno.

In [254]:
t=seq(0,10, by=0.05)
s = cos(2*pi*t) + sin(2*pi*t) + 0.5*cos(2*pi*2*t)
w=rnorm(length(t),0,0.2)
datos=s+w;

x_t = ts(datos,start=0,frequency=20)

tsplot(x_t, col = 8)


In [267]:
t1 = cos(2*pi*t)
t2 = sin(2*pi*t)
t3 = cos(2*2*pi*t)
t4 = sin(2*2*pi*t)
fit_0 = lm(x_t~ 0 +  t1 + t2, na.action=NULL)
fit_1 = lm(x_t~ 0 +  t1 + t2 + t3 + t4 , na.action=NULL)
fit_2 = lm(x_t~ 0 +  t1 + t2 + t3 , na.action=NULL)
summary (fit_0)
summary (fit_1)
summary (fit_2)


Call:
lm(formula = x_t ~ 0 + t1 + t2, na.action = NULL)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.98112 -0.28568  0.03552  0.33059  1.00221 

Coefficients:
   Estimate Std. Error t value Pr(>|t|)    
t1  0.98819    0.04007   24.66   <2e-16 ***
t2  0.95514    0.04027   23.72   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4027 on 199 degrees of freedom
Multiple R-squared:  0.8547,	Adjusted R-squared:  0.8532 
F-statistic: 585.3 on 2 and 199 DF,  p-value: < 2.2e-16



Call:
lm(formula = x_t ~ 0 + t1 + t2 + t3 + t4, na.action = NULL)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.60618 -0.10857  0.02363  0.15789  0.61114 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)    
t1  0.983417   0.021054  46.709   <2e-16 ***
t2  0.955141   0.021158  45.143   <2e-16 ***
t3  0.481850   0.021054  22.886   <2e-16 ***
t4 -0.009832   0.021158  -0.465    0.643    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2116 on 197 degrees of freedom
Multiple R-squared:  0.9603,	Adjusted R-squared:  0.9595 
F-statistic:  1191 on 4 and 197 DF,  p-value: < 2.2e-16



Call:
lm(formula = x_t ~ 0 + t1 + t2 + t3, na.action = NULL)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.60041 -0.10789  0.02218  0.15789  0.61692 

Coefficients:
   Estimate Std. Error t value Pr(>|t|)    
t1  0.98342    0.02101   46.80   <2e-16 ***
t2  0.95514    0.02112   45.23   <2e-16 ***
t3  0.48185    0.02101   22.93   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2112 on 198 degrees of freedom
Multiple R-squared:  0.9603,	Adjusted R-squared:  0.9597 
F-statistic:  1595 on 3 and 198 DF,  p-value: < 2.2e-16


In [268]:
predic_0 = ts(fitted(fit_0), start = 0, frequency = 20) #Creo una serie temporal con los valores ajustados. f = 1/0.05
tsplot(x_t, col = 8)
lines(predic_0, col = 3)

In [269]:
predic_1 = ts(fitted(fit_1), start = 0, frequency = 20) #Creo una serie temporal con los valores ajustados. f = 1/0.05
tsplot(x_t, col = 8)
lines(predic_1, col = 2)

In [275]:
predic_2 = ts(fitted(fit_2), start = 0, frequency = 20) #Creo una serie temporal con los valores ajustados. f = 1/0.05
tsplot(x_t, col = 8)
lines(predic_2, col = 4)

In [258]:
qqnorm(residuals(fit_0), col = 3)
qqline(residuals(fit_0), col = 8)

In [259]:
qqnorm(residuals(fit_1), col = 2)
qqline(residuals(fit_1), col = 8)

In [276]:
qqnorm(residuals(fit_2), col = 4)
qqline(residuals(fit_2), col = 8)

In [260]:
acf(residuals(fit_0), col = 3)

In [261]:
acf(residuals(fit_1), col = 2)

In [278]:
acf(residuals(fit_2), col = 4)

In [262]:
tsplot(residuals(fit_0), col = 3)

In [263]:
tsplot(residuals(fit_1), col = 2)

In [279]:
tsplot(residuals(fit_0), col = 4)

In [280]:
AIC(fit_0)
AIC(fit_1)
AIC(fit_2)


BIC(fit_0)
BIC(fit_1)
BIC(fit_2)

[1] 208.7905

[1] -47.99585

[1] -49.77564

[1] 218.7004

[1] -31.47932

[1] -36.56242

In [265]:
coef(fit_1)

t1           t2           t3           t4 
 0.983417341  0.955141428  0.481850441 -0.009832023

Despues de analizar los modelos planteados en el ejercicio, se puede observar que el modelo nùmero 2, se ajusta mejor que el modelo numero 1. Observando las graficas, en el modelo numero 1 los residuos aun se encuentran autocorrelacionados, a difernecia del modelo 2, el cual se puede observar que es ruido blanco gaussiano puro. Analizand el AIC y el BIC, se observa su mejorìa. 

De igual forma, analizanco los coeficientes del modelo, se podìa observr que en el modelo 2 existìa un valor muy insignificante con respecto a los demàs. para esto se ha recurrido a probar creando un nueo modelo "fit_2", en donde se remueve este valor. Dando como resultado un ajuste bastante similar al del modelo 2. Luego, por principio de parsimonia (Navaja de Ockham) y por los resultados generados por el AIC y el BIC, se puede decir que el modelo qu emejor se ajusta es el modelo "fit_2" que no cuenta con el coeficiente $\beta_5 \sin(2\pi 2t).$